In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bluff,-46.6000,168.3333,48.87,74,63,12.93,NZ,2021-11-17 16:26:27
1,1,Qaanaaq,77.4840,-69.3632,-2.52,73,12,5.53,GL,2021-11-17 16:26:27
2,2,Kapaa,22.0752,-159.3190,71.58,94,1,8.05,US,2021-11-17 16:26:27
3,3,Kathu,7.9178,98.3332,81.70,99,100,7.56,TH,2021-11-17 16:26:28
4,4,Bethel,41.3712,-73.4140,51.93,56,20,1.01,US,2021-11-17 16:26:28


In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])                          
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Kathu,7.9178,98.3332,81.70,99,100,7.56,TH,2021-11-17 16:26:28
14,14,Penha,-26.7694,-48.6458,83.55,78,40,12.66,BR,2021-11-17 16:26:31
20,20,Harper,4.3750,-7.7169,82.17,74,39,7.36,LR,2021-11-17 16:26:33
25,25,Dhrol,22.5667,70.4167,85.23,36,100,11.59,IN,2021-11-17 16:26:35
27,27,Maumere,-8.6199,122.2111,80.60,74,90,2.84,ID,2021-11-17 16:26:35
28,28,Fare,-16.7000,-151.0167,80.19,75,100,18.39,PF,2021-11-17 16:26:36
30,30,Jizan,17.3333,42.6667,85.14,62,4,9.53,SA,2021-11-17 16:26:36
32,32,Luxor,25.6989,32.6421,80.01,26,94,3.71,EG,2021-11-17 16:26:37
34,34,Hilo,19.7297,-155.0900,80.28,91,20,5.75,US,2021-11-17 16:26:37
38,38,Butaritari,3.0707,172.7902,81.61,76,85,12.10,KI,2021-11-17 16:26:38


In [12]:
#Add the following code to a new cell and run the cell to determine if there are any null values.
preferred_cities_df.count()

City_ID       99
City          99
Lat           99
Lng           99
Max Temp      99
Humidity      99
Cloudiness    99
Wind Speed    99
Country       98
Date          99
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Kathu,TH,81.70,7.9178,98.3332,
14,Penha,BR,83.55,-26.7694,-48.6458,
20,Harper,LR,82.17,4.3750,-7.7169,
25,Dhrol,IN,85.23,22.5667,70.4167,
27,Maumere,ID,80.60,-8.6199,122.2111,
28,Fare,PF,80.19,-16.7000,-151.0167,
30,Jizan,SA,85.14,17.3333,42.6667,
32,Luxor,EG,80.01,25.6989,32.6421,
34,Hilo,US,80.28,19.7297,-155.0900,
38,Butaritari,KI,81.61,3.0707,172.7902,


In [16]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [17]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Kathu,TH,81.70,7.9178,98.3332,Novotel Phuket Resort
14,Penha,BR,83.55,-26.7694,-48.6458,Hotel Mirante Do Parque
20,Harper,LR,82.17,4.3750,-7.7169,Screensaver
25,Dhrol,IN,85.23,22.5667,70.4167,Marknday Holiday
27,Maumere,ID,80.60,-8.6199,122.2111,Sylvia Hotel


In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Kathu,TH,81.70,7.9178,98.3332,Novotel Phuket Resort
14,Penha,BR,83.55,-26.7694,-48.6458,Hotel Mirante Do Parque
20,Harper,LR,82.17,4.3750,-7.7169,Screensaver
25,Dhrol,IN,85.23,22.5667,70.4167,Marknday Holiday
27,Maumere,ID,80.60,-8.6199,122.2111,Sylvia Hotel


In [21]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{City}</dd>
<dt>Another name</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))